# ANÁLISIS EXPLORATORIO DE DATOS

Importamos librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import requests as rq
import io

Obtenemos los datasets desde las URLs

In [ ]:
req_hosp_train = rq.get('https://raw.githubusercontent.com/soyHenry/Datathon/main/hospitalizaciones_train.csv').content

req_hosp_test = rq.get('https://raw.githubusercontent.com/soyHenry/Datathon/main/hospitalizaciones_test.csv').content

Convertimos el contenido obtenido anteriormente a DataFrames de Pandas para facilitar su manipulación

In [ ]:
df_hosp_train = pd.read_csv(io.StringIO(req_hosp_train.decode('utf-8')))

df_hosp_test = pd.read_csv(io.StringIO(req_hosp_test.decode('utf-8')))

In [ ]:
df_hosp_train.head(2)

In [ ]:
df_hosp_test.head(2)

Verificamos que la inexistencia de valores nulos

In [ ]:
df_hosp_train.info()

In [ ]:
df_hosp_test.info()

Verificamos la inexistencia de valores duplicados

In [ ]:
df_hosp_train.duplicated().sum()

In [ ]:
df_hosp_test.duplicated().sum()

Creamos nuestra columna objetivo a partir de la columna 'Stay (in days)' (en df_hosp_test agrego la misma columna pero vacía)

In [ ]:
df_hosp_train["extended_stay"] = df_hosp_train["Stay (in days)"].apply(lambda x: 1 if x > 8 else 0)

In [ ]:
df_hosp_test["pred"] = ''

Eliminamos aquellas columnas que como Científico de Datos considero prescindibles en el modelo 

In [ ]:
df_hosp_train.drop(columns=['patientid','Stay (in days)'], inplace=True)

In [ ]:
df_hosp_test.drop(columns=['patientid'], inplace=True)

Conversion de variables categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

df_hosp_train

In [ ]:
labelencoder_X = LabelEncoder()
df_hosp_train['Department'] = labelencoder_X.fit_transform(df_hosp_train['Department'])

labelencoder_X = LabelEncoder()
df_hosp_train['Ward_Facility_Code'] = labelencoder_X.fit_transform(df_hosp_train['Ward_Facility_Code'])

labelencoder_X = LabelEncoder()
df_hosp_train['doctor_name'] = labelencoder_X.fit_transform(df_hosp_train['doctor_name'])

labelencoder_X = LabelEncoder()
df_hosp_train['Age'] = labelencoder_X.fit_transform(df_hosp_train['Age'])

labelencoder_X = LabelEncoder()
df_hosp_train['gender'] = labelencoder_X.fit_transform(df_hosp_train['gender'])

labelencoder_X = LabelEncoder()
df_hosp_train['Type of Admission'] = labelencoder_X.fit_transform(df_hosp_train['Type of Admission'])

labelencoder_X = LabelEncoder()
df_hosp_train['Severity of Illness'] = labelencoder_X.fit_transform(df_hosp_train['Severity of Illness'])

labelencoder_X = LabelEncoder()
df_hosp_train['health_conditions'] = labelencoder_X.fit_transform(df_hosp_train['health_conditions'])

labelencoder_X = LabelEncoder()
df_hosp_train['Insurance'] = labelencoder_X.fit_transform(df_hosp_train['Insurance'])

In [ ]:
df_hosp_train

df_hosp_test

In [ ]:
labelencoder_X = LabelEncoder()
df_hosp_test['Department'] = labelencoder_X.fit_transform(df_hosp_test['Department'])

labelencoder_X = LabelEncoder()
df_hosp_test['Ward_Facility_Code'] = labelencoder_X.fit_transform(df_hosp_test['Ward_Facility_Code'])

labelencoder_X = LabelEncoder()
df_hosp_test['doctor_name'] = labelencoder_X.fit_transform(df_hosp_test['doctor_name'])

labelencoder_X = LabelEncoder()
df_hosp_test['Age'] = labelencoder_X.fit_transform(df_hosp_test['Age'])

labelencoder_X = LabelEncoder()
df_hosp_test['gender'] = labelencoder_X.fit_transform(df_hosp_test['gender'])

labelencoder_X = LabelEncoder()
df_hosp_test['Type of Admission'] = labelencoder_X.fit_transform(df_hosp_test['Type of Admission'])

labelencoder_X = LabelEncoder()
df_hosp_test['Severity of Illness'] = labelencoder_X.fit_transform(df_hosp_test['Severity of Illness'])

labelencoder_X = LabelEncoder()
df_hosp_test['health_conditions'] = labelencoder_X.fit_transform(df_hosp_test['health_conditions'])

labelencoder_X = LabelEncoder()
df_hosp_test['Insurance'] = labelencoder_X.fit_transform(df_hosp_test['Insurance'])

In [ ]:
df_hosp_test

# Ajustar el modelo en el Dataframe de Entrenamiento

Defino los valores de X e y

In [ ]:
X = df_hosp_train.iloc[:, :-1]
y = df_hosp_train.iloc[:,-1]

Dividimos en datos de entrenamiento y datos de testeo.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

Elegimos el modelo de K-Nearest Neighbors para resolver el problema de clasificación. Como parámetro elegimos 5 vecinos, la metrica minkowski y p=2 significa que elegimos la distancia manhattan para el modelo

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = "minkowski", p = 2)
classifier.fit(X_train, y_train)

In [ ]:
y_pred  = classifier.predict(X_test)

In [ ]:
y_pred

Matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

Calculo de Recall y Accuracy

In [ ]:
from sklearn import metrics

print("Recall: ", metrics.recall_score(y_test,y_pred))
print("Accuracy: ", metrics.accuracy_score(y_test,y_pred))


# Ajustar el modelo con el conjunto de Test

In [ ]:
X2 = df_hosp_test.iloc[:, :-1]

Evaluamos en modelo en el Dataframe de Test

In [ ]:
y_pred2  = classifier.predict(X2)

In [ ]:
y_pred2

Agregamos los valores predichos a la columna 'extended_stay' del Dataframe de Test

In [ ]:
df_hosp_test['pred'] = y_pred2

In [ ]:
df_hosp_test

Creamos un archivos csv con los datos que hemos predicho del Dataframe de de Test

In [ ]:
pd.DataFrame(y_pred2).to_csv('MatiasMartinez96.csv', index=False)